In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
from datasets import load_dataset

In [4]:
translation=load_dataset('SKNahin/bengali-transliteration-data')
translation.set_format(type='pandas')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [5]:
translation

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})

In [6]:
df=translation['train'][:]

In [7]:
df.isnull().sum()

,0
bn,0
rm,0


In [8]:
df.duplicated().sum()

202

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df

,bn,rm
0,স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???,scroll kore 20/30 second er video pann nai???
1,ও গুলা টরেন্ট সাইট এ পাবেন,o gula Torrent site e paben
2,ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই ...,vokkor chokkor post akta korlei approved…. nis...
3,আমি টেস্ট করেই কোড দিছি…,ami test koreii code disi…
4,"এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআ...","eto koster ki ache shakibwap.tk,shakibwap.ml&a..."
...,...,...
5001,এদের কে মাটি চাপা দেয়া দরকার,Eder ke mati chapa deya dorkar
5002,ওদের একদিন বিচার হবে ভাই,Oder ekdin bichar hobe vai
5003,অপেক্ষা করেন,Opekkha koren
5004,সময় আসবে,Shomoy ashbe


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "csebuetnlp/banglat5"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [12]:
def tokenize_data(input_texts, target_texts, tokenizer, max_length=128):
    if isinstance(input_texts, pd.Series):
        input_texts = input_texts.tolist()
    if isinstance(target_texts, pd.Series):
        target_texts = target_texts.tolist()

    inputs = tokenizer(input_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    targets = tokenizer(target_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    return inputs, targets

In [13]:
inputs, targets = tokenize_data(df['rm'], df['bn'], tokenizer)

In [14]:
inputs

{'input_ids': tensor([[14990,  2520,  2845,  ...,     0,     0,     0],
        [ 6522, 12378,  2267,  ...,     0,     0,     0],
        [23291,  1537,  1537,  ...,     0,     0,     0],
        ...,
        [ 2985,  5093,  1537,  ...,     0,     0,     0],
        [13412, 26267,    20,  ...,     0,     0,     0],
        [ 1037,  3030,  4814,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [15]:
targets

{'input_ids': tensor([[12008,  8727,     9,  ...,     0,     0,     0],
        [   11,  8832, 29499,  ...,     0,     0,     0],
        [ 2143,  6294,     7,  ...,     0,     0,     0],
        ...,
        [  720,    79,     1,  ...,     0,     0,     0],
        [   69,   616,     1,  ...,     0,     0,     0],
        [   74,    17,    69,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [16]:
from sklearn.model_selection import train_test_split
train_inputs, test_inputs, train_targets, test_targets = train_test_split(inputs['input_ids'], targets['input_ids'], test_size=0.2, random_state=42)

In [17]:
train_inputs.shape,test_inputs.shape,train_targets.shape,test_targets.shape

(torch.Size([3843, 96]),
 torch.Size([961, 96]),
 torch.Size([3843, 60]),
 torch.Size([961, 60]))

In [18]:
train_attention_mask, test_attention_mask, train_target_attention_mask, test_target_attention_mask = train_test_split(inputs['attention_mask'], targets['attention_mask'], test_size=0.2, random_state=42)

In [19]:
train_attention_mask.shape,test_attention_mask.shape,train_target_attention_mask.shape,test_target_attention_mask.shape

(torch.Size([3843, 96]),
 torch.Size([961, 96]),
 torch.Size([3843, 60]),
 torch.Size([961, 60]))

In [20]:
from torch.utils.data import DataLoader, Dataset

class TranslationDataset(Dataset):
    def __init__(self, inputs, targets, attention_masks, target_attention_masks):
        self.inputs = inputs
        self.targets = targets
        self.attention_masks = attention_masks
        self.target_attention_masks = target_attention_masks

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.inputs[idx]).contiguous(),
            "attention_mask": torch.tensor(self.attention_masks[idx]).contiguous(),
            "labels": torch.tensor(self.targets[idx]).contiguous(),
            "decoder_attention_mask": torch.tensor(self.target_attention_masks[idx]).contiguous(),
        }

In [21]:
train_dataset = TranslationDataset(
    train_inputs, train_targets, train_attention_mask, train_target_attention_mask)

In [22]:
train_dataset

In [23]:
val_dataset = TranslationDataset(
    test_inputs, test_targets, test_attention_mask, test_target_attention_mask
)

In [24]:
val_dataset

In [25]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [31]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# import wandb
ckpt_steps = 75
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    # save_steps=10_000,
    # save_total_limit=2,
    report_to='none',

    logging_steps=ckpt_steps,              # When to start reporting loss
    logging_dir="./logs",
    # save_steps = ckpt_steps,
    save_strategy= "steps"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

<ipython-input-32-1f5d2363040a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
import torch
trainer.train()

<ipython-input-20-3a0c71903ba5>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.inputs[idx]).contiguous(),
<ipython-input-20-3a0c71903ba5>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(self.attention_masks[idx]).contiguous(),
<ipython-input-20-3a0c71903ba5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(self.targets[idx]).contiguous(),
<ipython-input-20-3a0c71903ba5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourc

Step,Training Loss,Validation Loss
75,51.335200,36.068207
150,35.439200,19.929054
225,22.140200,8.201963


In [ ]:
# model.save_pretrained("./fine_tuned_model")
# tokenizer.save_pretrained("./fine_tuned_model")

In [ ]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model_path = "/kaggle/working/results/checkpoint-363"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# Example test dataset
test_inputs = ["Tumi kothay jacho?", "Ami valo achi."]
test_targets = ["তুমি কোথায় যাচ্ছ?", "আমি ভালো আছি।"]

predictions = []

for input_text in test_inputs:
    # Tokenize input text
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate predictions
    output_ids = model.generate(input_ids.cuda())

    # Decode the output
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predictions.append(decoded_output)

predictions